In [3]:
!pip install transformers
!pip install torch
!pip install -U PyPDF2
!pip install python-docx
!pip install pdfkit
!pip install textacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 53.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184505 sha256=a485f666386039281c1113631b5efc59bc3141fa9616c7c2ad4ef7da5d7f8887
  Stored in directory: /root/.cache/pip/wheels/8d/43/ab/6dfe2e7103b24fb1148e95b265fd71d23d29bcbaa60a4a7ed9
Successfully built python-docx
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.4/200.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 69.5 MB/s eta 0:00:00


In [5]:
!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10 
!jupyter notebook --NotebookApp.rate_limit_window=3 

[I 08:18:30.793 NotebookApp] Skipped non-installed server(s): bash-language-server, dockerfile-language-server-nodejs, javascript-typescript-langserver, jedi-language-server, julia-language-server, pyright, python-language-server, r-languageserver, sql-language-server, texlab, typescript-language-server, unified-language-server, vscode-css-languageserver-bin, vscode-html-languageserver-bin, vscode-json-languageserver-bin, yaml-language-server
[C 08:18:32.387 NotebookApp] Running as root is not recommended. Use --allow-root to bypass.
[I 08:18:34.489 NotebookApp] Skipped non-installed server(s): bash-language-server, dockerfile-language-server-nodejs, javascript-typescript-langserver, jedi-language-server, julia-language-server, pyright, python-language-server, r-languageserver, sql-language-server, texlab, typescript-language-server, unified-language-server, vscode-css-languageserver-bin, vscode-html-languageserver-bin, vscode-json-languageserver-bin, yaml-language-server
[C 08:18:36.5

In [7]:
import os
from PyPDF2 import PdfReader
import docx
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import pdfkit  
import re
import tqdm

In [ ]:
pdf_config_file = r"C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe"
config = pdfkit.configuration(wkhtmltopdf = pdf_config_file)  

In [34]:
article_links =[r"https://en.wikipedia.org/wiki/Artificial_intelligence", 
                r"https://www.brookings.edu/research/how-artificial-intelligence-is-transforming-the-world",
                r"https://www.zdnet.com/article/what-is-ai-heres-everything-you-need-to-know-about-artificial-intelligence/",
                r"https://www.pewresearch.org/internet/2018/12/10/artificial-intelligence-and-the-future-of-humans/",
                r"https://spectrum.ieee.org/topic/artificial-intelligence/",
                r"https://spectrum.ieee.org/topic/artificial-intelligence/",
                r"https://www.nytimes.com/article/ai-artificial-intelligence-chatbot.html",
                r"https://www.britannica.com/technology/artificial-intelligence",
                r"https://builtin.com/artificial-intelligence",
                r"https://www.ibm.com/topics/artificial-intelligence"]

In [8]:
print(os.getcwd())

/kaggle/working


In [37]:
def download_article(articles):
    if len(articles)>0:
        for link in tqdm.tqdm(articles):
            file_name = re.findall(r"https://\w+.(\w+).\w+",link)[0]+".pdf"
            out_file = os.path.join("dataset", file_name)
            pdfkit.from_url(link, out_file , configuration=config)
    else :
        print("not found links")

In [ ]:
from contextlib import contextmanager
import time
@contextmanager
def Timer():
    start = time.time()
    yield None
    end = time.time()
    print(f"Time used is {end-start:0.3f} secounds")

In [39]:
print("start Downloading...")
with Timer():
    download_article(article_links)
print("Downloading is finished!")

start Downloading...


100%|██████████| 10/10 [00:41<00:00,  4.17s/it]

Time used is 41.775 secounds



In [9]:
# Functions to read different file types
def read_pdf(file_path):
    with open(file_path, "rb") as file:
        pdf_reader = PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            text += pdf_reader.pages[page_num].extract_text()
    return text

def read_word(file_path):
    doc = docx.Document(file_path)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

def read_txt(file_path):
    with open(file_path, "r") as file:
        text = file.read()
    return text

def read_documents_from_directory(directory):
    combined_text = ""
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if filename.endswith(".pdf"):
            combined_text += read_pdf(file_path)
        elif filename.endswith(".docx"):
            combined_text += read_word(file_path)
        elif filename.endswith(".txt"):
            combined_text += read_txt(file_path)
    return combined_text



In [10]:
def clean_text(text):
  """Cleans text from noise such as any number or special character except ? , ., !, , and also remove specs and taps and any noise.

  Args:
    text: The text to be cleaned.

  Returns:
    The cleaned text.
  """

  # Remove any numbers from the text.
  text = re.sub(r'\d+', '', text)

  # Remove any special characters from the text except ? , ., !, ,.
  text = re.sub(r'[^\w\s?.!,]', '', text)

  # Remove any spaces from the beginning and end of the text.
  text = text.strip()

  # Remove any HTML tags from the text.
  text = re.sub(r'<[^>]+>', '', text)

  # Remove any links from the text.
  text = re.sub(r'https?://[^ ]+', '', text)

  # Remove any punctuation marks from the text.
  text = re.sub(r'[,!?.:;]', '', text)
   #remove bracket
  text = re.sub(r"\[[^\[\]]*\]", " ", text)
  text = re.sub(r"(?:^|\s)[&#<>{}\[\]+|\\:-]{1,}(?:\s|$)" , " ", text)
  text = re.sub(r"(?:^|\s)[\-=\+]{2,}(?:\s|$)"," ", text)
  text = re.sub(r"\s+" ," ", text)

  return text

#character normalization 
import textacy.preprocessing as tprep
def normalize(text):
    text = tprep.normalize.hyphenated_words(text)
    text = tprep.normalize.quotation_marks(text)
    text = tprep.normalize.unicode(text)
    text = tprep.remove.accents(text)
    return text 
def more_clean(text):
    text = tprep.replace.hashtags(text," ")
    text = tprep.replace.emojis(text , " ")
    text = tprep.remove.html_tags(text)
    text = tprep.replace.urls(text)
    return text    

def clean(text):
  text = clean_text(text)
  text = normalize(text)
  text = more_clean(text)
  return text


In [11]:
def train_chatbot(directory, model_output_path, train_fraction=0.8):
    # Read documents from the directory
    combined_text = read_documents_from_directory(directory)
    combined_text = re.sub(r'\n+', '\n', combined_text).strip()  # Remove excess newline characters

    # Split the text into training and validation sets
    split_index = int(train_fraction * len(combined_text))
    train_text = combined_text[:split_index]
    val_text = combined_text[split_index:]
    train_text = clean(train_text).encode("utf-8").strip()
    val_text = clean(val_text).encode("utf-8").strip()
    # Save the training and validation data as text files
    with open("train.txt", "wb") as f:
        f.write(train_text)
    with open("val.txt", "wb") as f:
        f.write(val_text)
    del combined_text, train_text, val_text
    
    print("start trainning ....")

    # Set up the tokenizer and model
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")  #also try gpt2, gpt2-large and  gpt2-xl
    model = GPT2LMHeadModel.from_pretrained("gpt2-medium")  #also try gpt2, gpt2-large and gpt2-large, also gpt2-xl

    # Prepare the dataset
    train_dataset = TextDataset(tokenizer=tokenizer, file_path="train.txt", block_size=128)
    val_dataset = TextDataset(tokenizer=tokenizer, file_path="val.txt", block_size=128)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False) #to make a padding 

    # Set up the training arguments
    training_args = TrainingArguments(
        output_dir=model_output_path,
        overwrite_output_dir=True,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=100,
        save_steps=10_000,
        save_total_limit=2,
        logging_dir='./logs',
    )

    # Train the model
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    trainer.train()
    trainer.save_model(model_output_path)
    
    # Save the tokenizer
    tokenizer.save_pretrained(model_output_path)


In [12]:
def generate_response(model, tokenizer, prompt, max_length=100):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [13]:
os.listdir("/kaggle/input/dataset")

['ARTIFICIAL INTELLIGENCEr.pdf',
 'EPRS_STU(2020)641547_EN.pdf',
 'com.pdf',
 'wikipedia.pdf',
 'zdnet.pdf',
 'ArtificialIntelligenceDefinitionEthicsandStandards.docx',
 'britannica.pdf',
 'artificial_intelligence_tutorial.pdf']

In [14]:
os.mkdir("model_output")

In [15]:

def main():
    directory = "/kaggle/input/dataset"  # Replace with the path to your directory containing the files
    model_output_path = "./model_output/"

    # Train the chatbot
    train_chatbot(directory, model_output_path)

    # Load the fine-tuned model and tokenizer
    model = GPT2LMHeadModel.from_pretrained(model_output_path)
    tokenizer = GPT2Tokenizer.from_pretrained(model_output_path)

    # Test the chatbot
    prompt = "what is Artificial intelligence?"  # Replace with your desired prompt
    response = generate_response(model, tokenizer, prompt)
    print("Generated response:", response)

In [16]:
if __name__ == "__main__":
    main()

start trainning ....


/opt/conda/lib/python3.7/site-packages/transformers/data/datasets/language_modeling.py:57: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  FutureWarning,
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,3.395900
1000,1.786000
1500,0.711700
2000,0.259600
2500,0.134600
3000,0.085800
3500,0.061600
4000,0.048900
4500,0.038700
5000,0.033400


Generated response: what is Artificial intelligence? What is machine learning Image Bing Image CreatorZDNET The biggest quality that sets AI aside from other computer science topics is the ability to easily automate tasks by employing machine learning which lets computers learn from different experiences rather than being explicitly programmed to perform each task This capability is what many refer to as AI but machine learning is actually a subset of artificial intelligence Machine learning involves a system being trained on large amounts of data so it can learn from mistakes and recognize patterns in order to accurately make


In [17]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel


In [18]:
def generate_response(model, tokenizer, prompt, max_length=250):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    
    # Create the attention mask and pad token id
    attention_mask = torch.ones_like(input_ids)
    pad_token_id = tokenizer.eos_token_id

    output = model.generate(
        input_ids,
        max_length=max_length,
        num_return_sequences=1,
        attention_mask=attention_mask,
        pad_token_id=pad_token_id
    )

    return tokenizer.decode(output[0], skip_special_tokens=True)


In [19]:
model_path = "/kaggle/working/model_output"
# Load the fine-tuned model and tokenizer
my_chat_model = GPT2LMHeadModel.from_pretrained(model_path)
my_chat_tokenizer = GPT2Tokenizer.from_pretrained(model_path)

In [20]:
prompt = "summarize what you learned about AI?"  # Replace with your desired prompt
#prompt = "What is the most promising future technology?"
response = generate_response(my_chat_model, my_chat_tokenizer, prompt, max_length=100)  #
print("Generated response:", response)

Generated response: summarize what you learned about AI? What are some recent examples of AI that you are most excited about and are looking forward to learning about next What are some of the most anticipated technologies in the next decade and what are some of the most anticipated technologies in the next decade What is machine learning and how does it work why does it matter and what can we do about it What is deep learning and how does it work why does it matter and what can we do about it What is fuzzy logic and


In [21]:
prompt = "what impact of artificial intelligience in market or socity or industry?"  # Replace with your desired prompt
#prompt = "What is the most promising future technology?"
response = generate_response(my_chat_model, my_chat_tokenizer, prompt, max_length=100)  #
print("Generated response:", response)

Generated response: what impact of artificial intelligience in market or socity or industry? What can we do about it Expert Group on Liability and New Technologies New Technologies Formation Liability for artificial intelligence and other emerging digital technologies European Commission High Level Expert Group on Arti ficial Intelligence Ethics guidelines for trustworthy AI European Commission High Level Expert Group on Artificial Intelligence Trustworthy AI assessment list European Commission High Level Expert Group on Artificial Intelligence Policy and investment recommendations for trustworthy AI European Commission Mazzucato M Mission oriented research i n


In [22]:
os.getcwd()

'/kaggle/working'

In [24]:
import shutil
shutil.make_archive("model_files", 'zip', "model_output")

'/kaggle/working/model_files.zip'

In [45]:
from IPython.display import FileLink
FileLink(r'model_files.zip')

/kaggle/working/model_files.zip